# Analysis of Taxi Trip Data

This notebook performs data analysis on taxi trip data, including data loading, transformation, and visualization. The steps include installing necessary packages, setting up display settings, connecting to the database, creating views for data analysis, and visualizing results.



##  Installation and Imports

 Install necessary packages and import libraries required for data manipulation and visualization.


In [ ]:
# Install DuckDB Libraries
!pip install -q duckdb

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import duckdb as db
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
from IPython.core.interactiveshell import InteractiveShell
from datetime import datetime, timedelta
import statsmodels.api as sm


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#  Display , Output and System Settings
Configure display settings for Pandas to ensure comprehensive visibility of data fields.

In [ ]:
# Change Pandas display settings to see all fields.


pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth', 400)
pd.set_option('display.max_columns', 100)
pd.set_option("display.precision", 5)
pd.set_option('display.width', 1000)


#-----------------------------------
# Modify the behavior of IPython/Jupyter notebook so that all expressions in a cell produce output, not just the last one.
# This can make it easier to follow and debug code by displaying intermediate results.


InteractiveShell.ast_node_interactivity = "all"

#----------------------------------
# Connect the database to memory for temporary data storing.

conn = db.connect(
    ':memory:'
)

# Modify database settings: memory limit/ see progress bar after running query/
# enable output explanation for all queries/ set timezone to New York. 

conn.sql('''
SET memory_limit = '25GB';
SET enable_progress_bar = true;
SET explain_output = 'all';
SET TimeZone='America/New_York';
'''
)

# See current database settings for parameters below:

conn.sql('''
SELECT *
FROM duckdb_settings()
WHERE name in ('TimeZone', 'Calendar', 'memory_limit', 'enable_progress_bar', 'explain_output', 'lock_configuration');
'''
)

# Print current time of the selected timezone

conn.sql('''
SELECT now()
'''
)

# Create Initial View
Define a view (tlctrip) combining trip data with location details for analysis.



In [ ]:
# Create a view of our dataset based on our Dictionary.


conn.sql('''
CREATE OR REPLACE VIEW init_tlctrip AS
SELECT 
    *,
    CASE 
        WHEN cap_type = 'green' THEN lpep_pickup_datetime
        WHEN cap_type = 'yellow' THEN tpep_pickup_datetime
        ELSE NULL
    END AS unified_pickup_datetime,
    CASE 
        WHEN cap_type = 'green' THEN lpep_dropoff_datetime
        WHEN cap_type = 'yellow' THEN tpep_dropoff_datetime
        ELSE NULL
    END AS unified_dropoff_datetime
FROM 
    read_parquet('/kaggle/input/tlc-trip-2020-01-to-2024-03/trips/*/*/*.parquet', hive_partitioning = true)
WHERE
    (
        unified_pickup_datetime < '2024-04-01' AND 
        unified_pickup_datetime >= '2020-01-01'
    ) AND
    (
        (fare_amount >= 0) AND
        (total_amount >= 0) AND
        ((extra >= 0) OR (extra IS NULL)) AND
        ((mta_tax >= 0) OR (mta_tax IS NULL)) AND
        ((tip_amount >= 0) OR (tip_amount IS NULL)) AND
        ((tolls_amount >= 0) OR (tolls_amount IS NULL)) AND
        ((ehail_fee >= 0) OR (ehail_fee IS NULL)) AND
        ((improvement_surcharge >= 0) OR (improvement_surcharge IS NULL)) AND
        ((congestion_surcharge >= 0) OR (congestion_surcharge IS NULL)) AND
        ((airport_fee >= 0) OR (airport_fee IS NULL))
    )
''')

# Create a view of NewYork init_taxizone dataset.

conn.sql('''
CREATE OR REPLACE VIEW init_taxizone AS 
SELECT * FROM 'https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv'
''')

# Take a look on our init_tlctrip view.

conn.sql('''
SELECT * FROM init_tlctrip LIMIT 2
''')

# Take a look on our init_taxizone view.

conn.sql('''
SELECT * FROM init_taxizone LIMIT 2
''')

# Get a look on fields data types.

conn.sql('''
DESCRIBE init_tlctrip
''').df()

conn.sql('''
DESCRIBE init_taxizone
''').df()


# **Q.1**
   * **1.1: Exploratory Analysis**
        * 1.1.1: Total Count
        * 1.1.2: Summary Statistics & Missing Values
        * 1.1.3: Detect fields timeline
        * 1.1.4: Validate Timestamps
        * 1.1.5: Cap Ratio
        * 1.1.6: Check Numeric Fields Values | Identify Outliers
        * 1.1.7: Assess Categorical Value Consistency

   * **1.2: Data Cleaning**
        * 1.2.1: Identify Duplicate Records
        * 1.2.2: Create Filtered View
        * 1.2.3: Exclude Duplicates
        * 1.2.4: Save Filtered_tlcTrip to a Parquet File
       
   * **1.3: Visualization**

## 1.1: Exploratory Analysis


###  1.1.1: Total Count

In [ ]:
# Take a look on the dataset size.

conn.sql('''
SELECT 
    COUNT(*)
FROM 
    init_tlctrip
''').to_df()
# Check init_taxizone count of records.

conn.sql('''
SELECT COUNT(*) FROM init_taxizone
''')

### 1.1.2: Summary Statistics & Missing Values

In [ ]:
# Get a look on fields statistical summary.

conn.sql('''
SUMMARIZE init_tlctrip
''').df()


### 1.1.3: Detect fields timeline

In [ ]:
# Check if all fields have data from 01-01-2020 to 31-03-2024.

conn.sql('''
SELECT min(unified_pickup_datetime) AS min_flag_datetime, max(unified_pickup_datetime) AS max_flag_datetime FROM init_tlctrip WHERE store_and_fwd_flag IS NOT NULL;
''')
conn.sql('''
SELECT min(unified_pickup_datetime) AS min_fare_datetime, max(unified_pickup_datetime) AS max_distance_datetime FROM init_tlctrip WHERE RatecodeID IS NOT NULL;
''')
conn.sql('''
SELECT min(unified_pickup_datetime) AS min_RatecodeID_datetime, max(unified_pickup_datetime) AS max_RatecodeID_datetime FROM init_tlctrip WHERE passenger_count IS NOT NULL;
''')
conn.sql('''
SELECT min(unified_pickup_datetime) AS min_congestion_datetime, max(unified_pickup_datetime) AS max_congestion_datetime FROM init_tlctrip WHERE congestion_surcharge IS NOT NULL;
''')
conn.sql('''
SELECT min(unified_pickup_datetime) AS min_airport_datetime, max(unified_pickup_datetime) AS max_airport_datetime FROM init_tlctrip WHERE airport_fee IS NOT NULL;
''')
conn.sql('''
SELECT min(unified_pickup_datetime) AS min_payment_datetime, max(unified_pickup_datetime) AS max_payment_datetime FROM init_tlctrip WHERE payment_type IS NOT NULL;
''')
conn.sql('''
SELECT min(unified_pickup_datetime) AS min_trip_datetime, max(unified_pickup_datetime) AS max_trip_datetime FROM init_tlctrip WHERE trip_type IS NOT NULL;
''')

### 1.1.4: Validate Timestamps


In [ ]:
# Check if there are any records with wrong timestamp

conn.sql('''
    SELECT 
        COUNT(*) AS invalid_timestamps
    FROM 
        init_tlctrip
    WHERE 
        unified_pickup_datetime::TIMESTAMPTZ >= unified_dropoff_datetime::TIMESTAMPTZ
''')

### 1.1.5: Cap Ratio

In [ ]:
# Check each 'cap_type' ratio to total.

conn.sql('''
    SELECT 
        cap_type, COUNT(*) AS cap_type_row, ROUND(cap_type_row*100/(SELECT COUNT(*) AS total_count FROM init_tlctrip), 2) AS percentage
    FROM 
        init_tlctrip
    GROUP BY
        1
''')

### 1.1.6: Check Numeric Fields Values | Identify Outliers

In [ ]:
# Check passenger_count values distribution.

conn.sql('''
    WITH passenger_bins AS (
        SELECT 
            FLOOR(passenger_count / 7) * 7 AS passenger_bin,
            COUNT(*) AS bin_count
        FROM 
            init_tlctrip
        WHERE 
            passenger_count IS NOT NULL
        GROUP BY 
            FLOOR(passenger_count / 7) * 7
    ),
    total_count AS (
        SELECT 
            COUNT(*) AS total
        FROM 
            init_tlctrip
        WHERE 
            passenger_count IS NOT NULL
    ),
    cumulative_bins AS (
        SELECT 
            pb.passenger_bin,
            pb.bin_count,
            SUM(pb.bin_count) OVER (ORDER BY pb.passenger_bin) AS cumulative_count
        FROM 
            passenger_bins pb
    )
    SELECT 
        CONCAT('0-', passenger_bin + 6) AS passenger_bin_range,
        bin_count,
        ROUND(cumulative_count * 100.0 / tc.total, 4) AS cumulative_percentage
    FROM 
        cumulative_bins cb, total_count tc
    ORDER BY 
        passenger_bin;
''')

In [ ]:
# Check trip_distance values distribution.

conn.sql('''
    WITH distance_bins AS (
        SELECT 
            FLOOR(trip_distance / 0.07) * 0.07 AS distance_bin,
            COUNT(*) AS bin_count
        FROM 
            init_tlctrip
        WHERE 
            trip_distance IS NOT NULL
            AND trip_distance > 0
        GROUP BY 
            FLOOR(trip_distance / 0.07) * 0.07
    ),
    total_count AS (
        SELECT 
            COUNT(*) AS total
        FROM 
            init_tlctrip
        WHERE 
            trip_distance IS NOT NULL
            AND trip_distance > 0
    ),
    cumulative_bins AS (
        SELECT 
            db.distance_bin,
            db.bin_count,
            SUM(db.bin_count) OVER (ORDER BY db.distance_bin) AS cumulative_count
        FROM 
            distance_bins db
    )
    SELECT 
        CONCAT('0-', distance_bin + 0.06) AS distance_bin_range,
        bin_count,
        ROUND(cumulative_count * 100.0 / tc.total, 4) AS cumulative_percentage
    FROM 
        cumulative_bins cb, total_count tc
    ORDER BY 
        distance_bin;
''')

In [ ]:
# Check speed values distribution.

conn.sql('''
WITH speed_bins AS (
    SELECT 
        ROUND(trip_distance / (EXTRACT(EPOCH FROM unified_dropoff_datetime) - EXTRACT(EPOCH FROM unified_pickup_datetime)) * 3600 / 22.5) * 22.5 AS speed_bin,
        COUNT(*) AS bin_count
    FROM 
        init_tlctrip
    WHERE 
        trip_distance > 0
        AND (EXTRACT(EPOCH FROM unified_dropoff_datetime) - EXTRACT(EPOCH FROM unified_pickup_datetime)) > 0
    GROUP BY 
        ROUND(trip_distance / (EXTRACT(EPOCH FROM unified_dropoff_datetime) - EXTRACT(EPOCH FROM unified_pickup_datetime)) * 3600 / 22.5) * 22.5
),
total_count AS (
    SELECT 
        COUNT(*) AS total
    FROM 
        init_tlctrip
    WHERE 
        trip_distance > 0
        AND (EXTRACT(EPOCH FROM unified_dropoff_datetime) - EXTRACT(EPOCH FROM unified_pickup_datetime)) > 0
),
cumulative_bins AS (
    SELECT 
        sb.speed_bin,
        sb.bin_count,
        SUM(sb.bin_count) OVER (ORDER BY sb.speed_bin) AS cumulative_count
    FROM 
        speed_bins sb
)
SELECT 
    CONCAT('0-', speed_bin + 22.5) AS speed_bin_range,
    bin_count,
    ROUND(cumulative_count * 100.0 / tc.total, 4) AS cumulative_percentage
FROM 
    cumulative_bins cb, total_count tc
ORDER BY 
    speed_bin;
''')

In [ ]:
# Check trip_duration values distribution.

conn.sql('''
    WITH trip_durations AS (
        SELECT
            (EXTRACT(EPOCH FROM unified_dropoff_datetime) - EXTRACT(EPOCH FROM unified_pickup_datetime)) / 60 AS trip_duration_minutes
        FROM
            init_tlctrip
        WHERE
            unified_dropoff_datetime IS NOT NULL
            AND unified_pickup_datetime IS NOT NULL
            AND unified_pickup_datetime < unified_dropoff_datetime
    ),
    duration_bins AS (
        SELECT
            FLOOR(trip_duration_minutes / 30) * 30 AS duration_bin,
            COUNT(*) AS bin_count
        FROM
            trip_durations
        GROUP BY
            FLOOR(trip_duration_minutes / 30) * 30
    ),
    total_count AS (
        SELECT
            COUNT(*) AS total
        FROM
            trip_durations
    ),
    cumulative_bins AS (
        SELECT
            db.duration_bin,
            db.bin_count,
            SUM(db.bin_count) OVER (ORDER BY db.duration_bin) AS cumulative_count
        FROM
            duration_bins db
    )
    SELECT
        CONCAT('0-', duration_bin + 29) AS tripduration_bin_range,
        bin_count,
        ROUND(cumulative_count * 100.0 / tc.total, 4) AS cumulative_percentage
    FROM
        cumulative_bins cb, total_count tc
    ORDER BY
        duration_bin
    LIMIT 10;
''')


In [ ]:
# Check fare_amount values distribution.

conn.sql('''
    WITH fare_bins AS (
    SELECT 
        FLOOR(fare_amount / 100) * 100 AS fare_bin,
        COUNT(*) AS bin_count
    FROM 
        init_tlctrip
    WHERE 
        fare_amount IS NOT NULL
    GROUP BY 
        FLOOR(fare_amount / 100) * 100
),
total_count AS (
    SELECT 
        COUNT(*) AS total
    FROM 
        init_tlctrip
    WHERE 
        fare_amount IS NOT NULL
),
cumulative_bins AS (
    SELECT 
        fb.fare_bin,
        fb.bin_count,
        SUM(fb.bin_count) OVER (ORDER BY fb.fare_bin) AS cumulative_count
    FROM 
        fare_bins fb
)
SELECT 
    CONCAT('0-', fare_bin + 99) AS fare_bin_range,
    bin_count,
    ROUND(cumulative_count * 100.0 / tc.total, 4) AS cumulative_percentage
FROM 
    cumulative_bins cb, total_count tc
ORDER BY 
    fare_bin;

''')


In [ ]:
# Check extra values distribution.

conn.sql('''
    WITH extra_bins AS (
        SELECT 
            FLOOR(extra / 10) * 10 AS extra_bin,
            COUNT(*) AS bin_count
        FROM 
            init_tlctrip
        WHERE 
            extra IS NOT NULL
        GROUP BY 
            FLOOR(extra / 10) * 10
    ),
    total_count AS (
        SELECT 
            COUNT(*) AS total
        FROM 
            init_tlctrip
        WHERE 
            extra IS NOT NULL
    ),
    cumulative_bins AS (
        SELECT 
            eb.extra_bin,
            eb.bin_count,
            SUM(eb.bin_count) OVER (ORDER BY eb.extra_bin) AS cumulative_count
        FROM 
            extra_bins eb
    )
    SELECT 
        CONCAT('0-', extra_bin + 9) AS extra_bin_range,
        bin_count,
        ROUND(cumulative_count * 100.0 / tc.total, 4) AS cumulative_percentage
    FROM 
        cumulative_bins cb, total_count tc
    ORDER BY 
        extra_bin;
''')

In [ ]:
# Check mta_tax values distribution.

conn.sql('''
    WITH mta_tax_bins AS (
        SELECT 
            FLOOR(mta_tax / 2) * 2 AS mta_tax_bin,
            COUNT(*) AS bin_count
        FROM 
            init_tlctrip
        WHERE 
            mta_tax IS NOT NULL
        GROUP BY 
            FLOOR(mta_tax / 2) * 2
    ),
    total_count AS (
        SELECT 
            COUNT(*) AS total
        FROM 
            init_tlctrip
        WHERE 
            mta_tax IS NOT NULL
    ),
    cumulative_bins AS (
        SELECT 
            mb.mta_tax_bin,
            mb.bin_count,
            SUM(mb.bin_count) OVER (ORDER BY mb.mta_tax_bin) AS cumulative_count
        FROM 
            mta_tax_bins mb
    )
    SELECT 
        CONCAT('0-', mta_tax_bin + 1) AS mta_bin_range,
        bin_count,
        ROUND(cumulative_count * 100.0 / tc.total, 4) AS cumulative_percentage
    FROM 
        cumulative_bins cb, total_count tc
    ORDER BY 
        mta_tax_bin;
''')

In [ ]:
# Check tip_amount values distribution.

conn.sql('''
    WITH tip_bins AS (
        SELECT 
            FLOOR(tip_amount / 10) * 10 AS tip_bin,
            COUNT(*) AS bin_count
        FROM 
            init_tlctrip
        WHERE 
            tip_amount IS NOT NULL
        GROUP BY 
            FLOOR(tip_amount / 10) * 10
    ),
    total_count AS (
        SELECT 
            COUNT(*) AS total
        FROM 
            init_tlctrip
        WHERE 
            tip_amount IS NOT NULL
    ),
    cumulative_bins AS (
        SELECT 
            tb.tip_bin,
            tb.bin_count,
            SUM(tb.bin_count) OVER (ORDER BY tb.tip_bin) AS cumulative_count
        FROM 
            tip_bins tb
    )
    SELECT 
        CONCAT('0-', tip_bin + 9) AS tip_bin_range,
        bin_count,
        ROUND(cumulative_count * 100.0 / tc.total, 4) AS cumulative_percentage
    FROM 
        cumulative_bins cb, total_count tc
    ORDER BY 
        tip_bin;
''')

In [ ]:
# Check tolls_amount values distribution.

conn.sql('''
    WITH tolls_bins AS (
        SELECT 
            FLOOR(tolls_amount / 10) * 10 AS tolls_bin,
            COUNT(*) AS bin_count
        FROM 
            init_tlctrip
        WHERE 
            tolls_amount IS NOT NULL
        GROUP BY 
            FLOOR(tolls_amount / 10) * 10
    ),
    total_count AS (
        SELECT 
            COUNT(*) AS total
        FROM 
            init_tlctrip
        WHERE 
            tolls_amount IS NOT NULL
    ),
    cumulative_bins AS (
        SELECT 
            tb.tolls_bin,
            tb.bin_count,
            SUM(tb.bin_count) OVER (ORDER BY tb.tolls_bin) AS cumulative_count
        FROM 
            tolls_bins tb
    )
    SELECT 
        CONCAT('0-', tolls_bin + 9) AS tolls_bin_range,
        bin_count,
        ROUND(cumulative_count * 100.0 / tc.total, 4) AS cumulative_percentage
    FROM 
        cumulative_bins cb, total_count tc
    ORDER BY 
        tolls_bin;
''')

In [ ]:
# Check improvement_surchage values distribution.

conn.sql('''
    WITH improvement_bins AS (
        SELECT 
            FLOOR(improvement_surcharge / 0.1) * 0.1 AS improvement_bin,
            COUNT(*) AS bin_count
        FROM 
            init_tlctrip
        WHERE 
            improvement_surcharge IS NOT NULL
        GROUP BY 
            FLOOR(improvement_surcharge / 0.1) * 0.1
    ),
    total_count AS (
        SELECT 
            COUNT(*) AS total
        FROM 
            init_tlctrip
        WHERE 
            improvement_surcharge IS NOT NULL
    ),
    cumulative_bins AS (
        SELECT 
            ib.improvement_bin,
            ib.bin_count,
            SUM(ib.bin_count) OVER (ORDER BY ib.improvement_bin) AS cumulative_count
        FROM 
            improvement_bins ib
    )
    SELECT 
        CONCAT('0-', improvement_bin + 0.099) AS improvement_bin_range,
        bin_count,
        ROUND(cumulative_count * 100.0 / tc.total, 4) AS cumulative_percentage
    FROM 
        cumulative_bins cb, total_count tc
    ORDER BY 
        improvement_bin;
''')

In [ ]:
# Check total_amount values distribution.

conn.sql('''
    WITH total_amount_bins AS (
        SELECT 
            FLOOR(total_amount / 50) * 50 AS total_amount_bin,
            COUNT(*) AS bin_count
        FROM 
            init_tlctrip
        WHERE 
            total_amount IS NOT NULL
        GROUP BY 
            FLOOR(total_amount / 50) * 50
    ),
    total_count AS (
        SELECT 
            COUNT(*) AS total
        FROM 
            init_tlctrip
        WHERE 
            total_amount IS NOT NULL
    ),
    cumulative_bins AS (
        SELECT 
            tab.total_amount_bin,
            tab.bin_count,
            SUM(tab.bin_count) OVER (ORDER BY tab.total_amount_bin) AS cumulative_count
        FROM 
            total_amount_bins tab
    )
    SELECT 
        CONCAT('0-', total_amount_bin + 49) AS total_bin_range,
        bin_count,
        ROUND(cumulative_count * 100.0 / tc.total, 4) AS cumulative_percentage
    FROM 
        cumulative_bins cb, total_count tc
    ORDER BY 
        total_amount_bin;
''')

In [ ]:
# Check congestion_surcharge values distribution.

conn.sql('''
    WITH congestion_bins AS (
        SELECT 
            FLOOR(congestion_surcharge / 0.25) * 0.25 AS congestion_bin,
            COUNT(*) AS bin_count
        FROM 
            init_tlctrip
        WHERE 
            congestion_surcharge IS NOT NULL
        GROUP BY 
            FLOOR(congestion_surcharge / 0.25) * 0.25
    ),
    total_count AS (
        SELECT 
            COUNT(*) AS total
        FROM 
            init_tlctrip
        WHERE 
            congestion_surcharge IS NOT NULL
    ),
    cumulative_bins AS (
        SELECT 
            cb.congestion_bin,
            cb.bin_count,
            SUM(cb.bin_count) OVER (ORDER BY cb.congestion_bin) AS cumulative_count
        FROM 
            congestion_bins cb
    )
    SELECT 
        CONCAT('0-', congestion_bin + 0.249) AS congestion_bin_range,
        bin_count,
        ROUND(cumulative_count * 100.0 / tc.total, 4) AS cumulative_percentage
    FROM 
        cumulative_bins cb, total_count tc
    ORDER BY 
        congestion_bin;
''')

In [ ]:
# Check airport_fee values distribution.

conn.sql('''
    WITH airport_fee_bins AS (
        SELECT 
            FLOOR(airport_fee / 0.25) * 0.25 AS airport_fee_bin,
            COUNT(*) AS bin_count
        FROM 
            init_tlctrip
        WHERE 
            airport_fee IS NOT NULL
        GROUP BY 
            FLOOR(airport_fee / 0.25) * 0.25
    ),
    total_count AS (
        SELECT 
            COUNT(*) AS total
        FROM 
            init_tlctrip
        WHERE 
            airport_fee IS NOT NULL
    ),
    cumulative_bins AS (
        SELECT 
            afb.airport_fee_bin,
            afb.bin_count,
            SUM(afb.bin_count) OVER (ORDER BY afb.airport_fee_bin) AS cumulative_count
        FROM 
            airport_fee_bins afb
    )
    SELECT 
        CONCAT('0-', airport_fee_bin + 0.249) AS airport_bin_range,
        bin_count,
        ROUND(cumulative_count * 100.0 / tc.total, 4) AS cumulative_percentage
    FROM 
        cumulative_bins cb, total_count tc
    ORDER BY 
        airport_fee_bin;
''')

### 1.1.7: Assess Categorical Value Consistency

In [ ]:
# Check if there is any odd value for 'store_and_fwd_flag' except N & Y.

conn.sql('''
    SELECT 
        DISTINCT store_and_fwd_flag, COUNT(*) AS count, ROUND(COUNT(*)*100/(SELECT COUNT('*') FROM init_tlctrip), 2) AS percentage
    FROM 
        init_tlctrip
    GROUP BY 
        1
    ORDER BY 
        1
''').to_df()
# Check if there is any odd value for 'payment_type' except 1 to 6.

conn.sql('''
    SELECT 
        payment_type, COUNT(*), ROUND(COUNT(*)*100/(SELECT COUNT('*') FROM init_tlctrip), 2) AS percentage
    FROM 
        init_tlctrip
    GROUP BY 
        1
    ORDER BY 
        1
''').to_df()
# Check if there is any odd value for 'VendorID' except 1 & 2.

conn.sql('''
    SELECT 
        cap_type, VendorID, COUNT(*), ROUND(COUNT(*)*100/(SELECT COUNT('*') FROM init_tlctrip), 2) AS percentage
    FROM 
        init_tlctrip
    GROUP BY 
        1, 2
    ORDER BY 
        1, 2
''').to_df()
# Check if there is any odd value for 'RatecodeID' except 1 to 6.

conn.sql('''
    SELECT 
        RatecodeID, COUNT(*), ROUND(COUNT(*)*100/(SELECT COUNT('*') FROM init_tlctrip), 2) AS percentage
    FROM 
        init_tlctrip
    GROUP BY 
        1
    ORDER BY
        1
''').to_df()
# Check if there is any odd value for 'trip_type' except 1 or 2.

conn.sql('''
    SELECT 
        cap_type, trip_type, COUNT(*), ROUND(COUNT(*)*100/(SELECT COUNT('*') FROM init_tlctrip), 2) AS percentage
    FROM 
        init_tlctrip
    GROUP BY 
        1, 2
    ORDER BY
        1, 2
''').to_df()



##  1.2: Data Cleaning

### 1.2.1: Identify Duplicate Records


In [ ]:
# Check if there are duplicates.

conn.sql('''
    SELECT 
        VendorID, unified_pickup_datetime, unified_dropoff_datetime, trip_distance, PULocationID, DOLocationID, COUNT(*) AS number_of_rows, SUM(number_of_rows) OVER ()
    FROM 
        init_tlctrip
    GROUP BY 
        VendorID, unified_pickup_datetime, unified_dropoff_datetime, trip_distance, PULocationID, DOLocationID
    HAVING 
        COUNT(*) > 1
    ORDER BY unified_pickup_datetime,
            unified_dropoff_datetime,
            trip_distance,
            PULocationID,
            DOLocationID 
''')

### 1.2.2: Create Filtered View

In [ ]:
# Create a new view due to outliers and out of range data. 

conn.sql('''
CREATE OR REPLACE VIEW filtered_tlctrip AS
WITH main_tlctrip AS (
    SELECT 
        CAST(VendorID AS INT) AS VendorID,
        CASE 
            WHEN store_and_fwd_flag = 'Y' THEN true
            WHEN store_and_fwd_flag = 'N' THEN false
            ELSE NULL
        END AS store_and_fwd_flag,
        CAST(RatecodeID AS INT) AS RatecodeID,
        CAST(PULocationID AS INT) AS PULocationID,
        CAST(DOLocationID AS INT) AS DOLocationID,
        passenger_count,
        trip_distance,
        fare_amount,
        extra,
        mta_tax,
        tip_amount,
        tolls_amount,
        improvement_surcharge,
        total_amount,
        payment_type,
        trip_type,
        congestion_surcharge,
        airport_fee,
        cap_type,
        CAST(year AS INT) AS year,
        CASE 
            WHEN cap_type = 'green' THEN lpep_pickup_datetime
            WHEN cap_type = 'yellow' THEN tpep_pickup_datetime
            ELSE NULL
        END AS unified_pickup_datetime,
        CASE 
            WHEN cap_type = 'green' THEN lpep_dropoff_datetime
            WHEN cap_type = 'yellow' THEN tpep_dropoff_datetime
            ELSE NULL
        END AS unified_dropoff_datetime
    FROM 
        read_parquet('/kaggle/input/tlc-trip-2020-01-to-2024-03/trips/*/*/*.parquet', hive_partitioning = true)
    WHERE
        unified_pickup_datetime < '2024-04-01' AND 
        unified_pickup_datetime >= '2020-01-01' AND
        unified_pickup_datetime::TIMESTAMPTZ < unified_dropoff_datetime::TIMESTAMPTZ AND
        payment_type NOT IN (0, 6) AND
        VendorID NOT IN (5, 6) AND
        RatecodeID != 99 AND
        (fare_amount BETWEEN 0 AND 99 OR fare_amount IS NULL) AND
        (extra BETWEEN 0 AND 10 OR extra IS NULL) AND
        (mta_tax BETWEEN 0 AND 3 OR mta_tax IS NULL) AND
        (tip_amount BETWEEN 0 AND 20 OR tip_amount IS NULL) AND
        (tolls_amount BETWEEN 0 AND 20 OR tolls_amount IS NULL) AND
        (passenger_count BETWEEN 1 AND 6 OR passenger_count IS NULL) AND
        trip_distance > 0 AND
        trip_distance <= 500 AND
        trip_distance / (EXTRACT(EPOCH FROM unified_dropoff_datetime) - EXTRACT(EPOCH FROM unified_pickup_datetime)) * 3600 <= 90 AND
        fare_amount >= 0 AND
        total_amount >= 0 AND
        extra >= 0 AND
        mta_tax >= 0 AND
        tip_amount >= 0 AND
        tolls_amount >= 0 AND
        (improvement_surcharge >= 0 OR improvement_surcharge IS NULL) AND
        (congestion_surcharge >= 0 OR congestion_surcharge IS NULL) AND
        (airport_fee >= 0 OR airport_fee IS NULL) AND
        trip_distance > 0.31 AND
        ((EXTRACT(EPOCH FROM unified_dropoff_datetime) - EXTRACT(EPOCH FROM unified_pickup_datetime)) > 60)
)
SELECT 
    tlc.VendorID,
    tlc.store_and_fwd_flag,
    tlc.RatecodeID,
    tlc.PULocationID,
    tlc.DOLocationID,
    tlc.passenger_count,
    tlc.trip_distance,
    tlc.fare_amount,
    tlc.extra,
    tlc.mta_tax,
    tlc.tip_amount,
    tlc.tolls_amount,
    tlc.improvement_surcharge,
    tlc.total_amount,
    tlc.payment_type,
    tlc.trip_type,
    tlc.congestion_surcharge,
    tlc.airport_fee,
    tlc.cap_type,
    tlc.year,
    tlc.unified_pickup_datetime,
    tlc.unified_dropoff_datetime,
    taxi.Zone AS PUZone,
    CASE 
        WHEN tlc.DOLocationID = taxi.LocationID THEN taxi.Zone 
        ELSE (
            SELECT Zone 
            FROM 'https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv' 
            WHERE LocationID = tlc.DOLocationID
        ) 
    END AS DOZone
FROM 
    main_tlctrip AS tlc
LEFT JOIN (
    SELECT 
        LocationID, 
        Zone 
    FROM 
        'https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv'
) AS taxi
ON 
    tlc.PULocationID = taxi.LocationID;
''')


conn.sql('''
DESCRIBE filtered_tlctrip
''')

In [ ]:
# Check filtered_tlctrip records count.

conn.sql('''
SELECT COUNT(*) FROM filtered_tlctrip
''')

### 1.2.3: Exclude Duplicates

In [ ]:
# Check if there are duplicates.

conn.sql('''
    SELECT 
        VendorID, unified_pickup_datetime, unified_dropoff_datetime, trip_distance, PULocationID, DOLocationID, COUNT(*) AS number_of_rows, SUM(number_of_rows) OVER ()
    FROM 
        filtered_tlctrip
    GROUP BY 
        VendorID, unified_pickup_datetime, unified_dropoff_datetime, trip_distance, PULocationID, DOLocationID
    HAVING 
        COUNT(*) > 1
    ORDER BY unified_pickup_datetime,
            unified_dropoff_datetime,
            trip_distance,
            PULocationID,
            DOLocationID 
''')

In [ ]:
# Create a new view without duplicates (there are only 52 rows that must be excluded).

conn.sql('''
CREATE OR REPLACE VIEW clean_tlctrip AS
WITH ranked_trips AS (
    SELECT
        *,
        ROW_NUMBER() OVER (PARTITION BY VendorID, unified_pickup_datetime, unified_dropoff_datetime, trip_distance, PULocationID, DOLocationID ORDER BY unified_pickup_datetime) AS rn
    FROM
        filtered_tlctrip
)
SELECT
    VendorID,
    store_and_fwd_flag,
    RatecodeID,
    PULocationID,
    DOLocationID,
    passenger_count,
    trip_distance,
    fare_amount,
    extra,
    mta_tax,
    tip_amount,
    tolls_amount,
    improvement_surcharge,
    total_amount,
    payment_type,
    trip_type,
    congestion_surcharge,
    airport_fee,
    cap_type,
    year,
    unified_pickup_datetime,
    unified_dropoff_datetime,
    PUZone,
    DOZone, 
FROM
    ranked_trips
WHERE
    rn = 1;
''')

In [ ]:
# Check tlctrip records count after excluding duplicates.

conn.sql('''
SELECT COUNT(*) FROM clean_tlctrip
''')

### 1.2.4: Save Filtered_tlcTrip to a Parquet File

In [ ]:
# # Define the path where you want to save the Parquet file
output_parquet_path = '/kaggle/working/tlctrip3.parquet'

 # Execute the SQL query to create and save the Parquet file directly
conn.sql(f'''
 COPY (SELECT * FROM filtered_tlctrip) TO '{output_parquet_path}' (FORMAT PARQUET);
 ''')



In [ ]:
conn.sql('''
CREATE OR REPLACE VIEW tlctrip AS
SELECT 
    *
FROM 
    read_parquet('/kaggle/input/final-tlctrip-zone-view/tlctrip3.parquet', hive_partitioning = true)
''')

In [ ]:
# Check tlctrip records count.

conn.sql('''
SELECT COUNT(*) FROM tlctrip
''')

In [ ]:
# Get a look on fields statistical summary.

conn.sql('''
SUMMARIZE tlctrip
''').df()

## 1.3: Visualization

In [ ]:
# Run the SQL query and print the resulting DataFrame
df = conn.sql('''
SELECT 
    date_trunc('month', unified_pickup_datetime) as unified_pickup_month,
    cap_type,
    COUNT(*) AS trip_count,
    AVG(fare_amount/trip_distance) AS USD_per_mile,
    AVG(fare_amount/(trip_distance*passenger_count)) AS USD_per_mile_person,
    AVG(total_amount) as avg_total_amount
FROM 
    tlctrip
GROUP BY
    1, 2
ORDER BY
    1, 2
''').df()

# Check if the DataFrame is not empty
print(df.head())

# Define the color map for the lines
color_map = {'green': 'green', 'yellow': 'darkgoldenrod'}

# Ensure the DataFrame is not empty
if not df.empty:
    # Convert the unified_pickup_month to datetime for proper plotting
    df['unified_pickup_month'] = pd.to_datetime(df['unified_pickup_month'])

    # Plot each figure with updated axis labels
    fig = px.line(df, x='unified_pickup_month', y='trip_count', color='cap_type', title='Trip Count per Month', 
                  color_discrete_map=color_map)
    fig.update_xaxes(title_text='Date', dtick="M4", tickformat="%b %Y")  # Update x-axis to show every 3 months
    fig.update_yaxes(title_text='Trip Count')
    fig.show()

    fig = px.line(df, x='unified_pickup_month', y='avg_total_amount', color='cap_type', title='Average Total Amount', 
                  color_discrete_map=color_map)
    fig.update_xaxes(title_text='Date', dtick="M4", tickformat="%b %Y")  # Update x-axis to show every 3 months
    fig.update_yaxes(title_text='Average Total Amount ($)')
    fig.show()

    fig = px.line(df, x='unified_pickup_month', y='USD_per_mile', color='cap_type', title='Fare Amount per Mile', 
                  color_discrete_map=color_map)
    fig.update_xaxes(title_text='Date', dtick="M4", tickformat="%b %Y")  # Update x-axis to show every 3 months
    fig.update_yaxes(title_text='Cost per Mile ($)')
    fig.show()

    fig = px.line(df, x='unified_pickup_month', y='USD_per_mile_person', color='cap_type', title='Fare Amount per Mile-Person', 
                  color_discrete_map=color_map)
    fig.update_xaxes(title_text='Date', dtick="M4", tickformat="%b %Y")  # Update x-axis to show every 3 months
    fig.update_yaxes(title_text='Cost per Mile Person ($)')
    fig.show()
else:
    print("The DataFrame is empty.")


In [ ]:
# Run the SQL query and print the resulting DataFrame
df = conn.sql('''
SELECT 
    date_trunc('month', unified_pickup_datetime) as unified_pickup_month,
    cap_type,
    COUNT(*) AS trip_count,
    AVG(fare_amount/trip_distance) AS USD_per_mile,
    AVG(fare_amount/(trip_distance*passenger_count)) AS USD_per_mile_person,
    AVG(total_amount) as avg_total_amount
FROM 
    tlctrip
GROUP BY
    1, 2
ORDER BY
    1, 2
''').df()

# Check if the DataFrame is not empty
print(df.head())

# Define the color map for the lines
color_map = {'green': 'green', 'yellow': 'darkgoldenrod'}

# Ensure the DataFrame is not empty
if not df.empty:
    # Convert the unified_pickup_month to datetime for proper plotting
    df['unified_pickup_month'] = pd.to_datetime(df['unified_pickup_month'])

    # Plot each figure with updated axis labels
    fig = px.line(df, x='unified_pickup_month', y='trip_count', color='cap_type', title='Trip Count per Month', 
                  color_discrete_map=color_map)
    fig.update_xaxes(title_text='Date', dtick="M4", tickformat="%b %Y")  # Update x-axis to show every 3 months
    fig.update_yaxes(title_text='Trip Count')
    fig.show()

    fig = px.line(df, x='unified_pickup_month', y='avg_total_amount', color='cap_type', title='Average Total Amount', 
                  color_discrete_map=color_map)
    fig.update_xaxes(title_text='Date', dtick="M4", tickformat="%b %Y")  # Update x-axis to show every 3 months
    fig.update_yaxes(title_text='Average Total Amount ($)')
    fig.show()

    fig = px.line(df, x='unified_pickup_month', y='USD_per_mile', color='cap_type', title='Fare Amount per Mile', 
                  color_discrete_map=color_map)
    fig.update_xaxes(title_text='Date', dtick="M4", tickformat="%b %Y")  # Update x-axis to show every 3 months
    fig.update_yaxes(title_text='Cost per Mile ($)')
    fig.show()

    fig = px.line(df, x='unified_pickup_month', y='USD_per_mile_person', color='cap_type', title='Fare Amount per Mile-Person', 
                  color_discrete_map=color_map)
    fig.update_xaxes(title_text='Date', dtick="M4", tickformat="%b %Y")  # Update x-axis to show every 3 months
    fig.update_yaxes(title_text='Cost per Mile Person ($)')
    fig.show()
else:
    print("The DataFrame is empty.")


In [ ]:
# Take a look on payment_type distribution.
# Example SQL query using DuckDB
query = '''
    SELECT payment_type, COUNT(*) as count, (COUNT(*) * 100.0 / (SELECT COUNT(*) FROM tlctrip)) AS percentage
    FROM tlctrip
    GROUP BY payment_type
'''

# Connect to DuckDB and execute the query
df = conn.execute(query).fetchdf()

# Convert query result to a pandas DataFrame
df.columns = ['payment_type', 'count', 'percentage']

# Save data to CSV file
df[['payment_type', 'percentage']].to_csv('payment_type_percentages.csv', index=False)

# Display the DataFrame (optional)
print(df)

# **Q.2**
   * **2.1: Selecting Data**
        
   * **2.2: Market Share Chart**
   
   * **2.3: Calculating Profit**
   
   * **2.4: Calculating Profit**
   
   * **2.4: Calculation of market volume in previous years**
   
   * **2.5: Graph of market volume growth rate in recent years**
   
   

## 2.1: Selecting Data

In [ ]:
conn.sql('''
CREATE OR REPLACE VIEW tlctrip AS 
SELECT *
FROM '/kaggle/input/final-tlctrip-zone-view/tlctrip3.parquet'
''')
conn.sql('''
CREATE OR REPLACE VIEW new_tlctrip AS
SELECT 
     PUZone as zone
    ,PUlocationID AS start_location_id 
    ,count(*) AS number_of_rides
    ,sum(total_amount) AS sum_of_price
    ,percentile_disc(0.50) WITHIN GROUP (ORDER BY total_amount) AS price_quartile_50
    ,sum_of_price/(SELECT SUM(total_amount) FROM tlctrip WHERE unified_pickup_datetime BETWEEN '2023-04-01 00:00:00' AND '2024-03-31 00:00:00' AND payment_type=1)*100 AS market_cap_percentage
    ,number_of_rides * price_quartile_50 AS metric
FROM tlctrip
WHERE 
    unified_pickup_datetime BETWEEN '2023-04-01 00:00:00' AND '2024-03-31 00:00:00'  
    AND payment_type=1
GROUP BY 1,2
ORDER BY 7 desc
''')

In [ ]:
df21 = conn.sql('''
select sum(market_cap_percentage) from new_tlctrip
''').to_df()
df21

In [ ]:
df22 = conn.sql('''
SELECT 
*
,SUM(market_cap_percentage) OVER (ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cumulative_market_cap_percentage
FROM new_tlctrip
LIMIT 12
''').to_df()
df22

## 2.2:Market share chart

In [ ]:
df_sorted = df22.sort_values(by='market_cap_percentage', ascending=True)

# Create the horizontal bar chart using px.bar
fig = px.bar(
    df_sorted,  # Data source (sorted DataFrame)
    x='market_cap_percentage',  # x-axis (market_cap_percentage)
    y='zone',  # y-axis (zone)
    title='Market Capacity Percentage by Zone In a Recent Year',
    orientation='h',  # Horizontal orientation
    #color='zone',  # Color based on market_cap_percentage
    text=df_sorted['market_cap_percentage'].round(1),  # Display values as text on bars
    labels={  # Customize axis labels
        'market_cap_percentage': 'Market Cap Percentage (%)',
        'zone': 'Zone'
    }
)

# Display the chart
fig.show()

## 2.3:Calculating Profit

In [ ]:
df23 = conn.sql('''
SELECT 
    SUM(total_amount)*(52/100)*(10/100)*(1.5/100) AS benefit
FROM tlctrip 
WHERE 
    unified_pickup_datetime BETWEEN '2023-04-01 00:00:00' AND '2024-03-31 00:00:00'  
    AND payment_type=1
''').to_df()
df23

## 2.4:Calculation of market volume in previous years

In [ ]:
df24 = conn.sql('''
SELECT 
    Case
        WHEN unified_pickup_datetime BETWEEN '2023-04-01 00:00:00' AND '2024-04-01 00:00:00' THEN '2023-2024'
        WHEN unified_pickup_datetime BETWEEN '2022-04-01 00:00:00' AND '2023-03-31 00:00:00' THEN '2022-2023'
        WHEN unified_pickup_datetime BETWEEN '2021-04-01 00:00:00' AND '2022-03-31 00:00:00' THEN '2021-2022'
        WHEN unified_pickup_datetime BETWEEN '2020-04-01 00:00:00' AND '2021-03-31 00:00:00' THEN '2020-2021'
    ELSE '0'
    END AS year_period
    ,Round(SUM(total_amount) ,1) AS market_cap
    ,(market_cap - LAG(market_cap) OVER (ORDER BY year_period)) / LAG(market_cap) OVER (ORDER BY year_period) * 100 AS market_cap_growth_rate_percentage
FROM tlctrip 
WHERE 
     payment_type = 1
     AND year_period != '0'
GROUP BY 1
order by 1 
''').to_df()
df24

## Graph of market volume growth rate in recent years

In [ ]:
df24 = df24.sort_values(by='market_cap', ascending=True)
fig = fig = px.bar(
    data_frame=df24,  # Data source (DataFrame)
    x='year_period',  # x-axis (year_period)
    y='market_cap',  # y-axis (market_cap)
    title='Market Capacity In Recent Years',
    text='market_cap',  # Display values as text on bars
    labels={  # Customize axis labels
        'market_cap': 'Market Capacity',
        'year_period': 'Year Period'
    }
)
fig.show()

In [ ]:
# Extract data for each plot and save to CSV files
plot_data = {
    'Market Capacity In Recent Years': df24[['year_period', 'market_cap']],
    'Market Capacity Percentage by Zone In a Recent Year': df22[['zone', 'market_cap_percentage']],
}

for title, data in plot_data.items():
    csv_filename = title.replace(' ', '_').lower() + '.csv'
    data.to_csv(csv_filename, index=False)
    print(f'Data for "{title}" saved to {csv_filename}')
else:
    print("The DataFrame is empty.")

# **Q.3**
   * **3.1: Annual Duration Times Passengers**

   * **3.2: Calculate the Duration Times Passengers for the Top 5 Zones**

## 3.1: Annual Duration Times Passengers

In [ ]:
# Calculate the Annual Duration Times Passengers (DTP), tlcTrip revenue & advertising company gross revenue for the last 12 months.

conn.sql('''
WITH trip_details AS (
  SELECT
    EXTRACT(EPOCH FROM unified_dropoff_datetime) - EXTRACT(EPOCH FROM unified_pickup_datetime) AS trip_duration_seconds,
    passenger_count
  FROM
    tlctrip
  WHERE unified_pickup_datetime BETWEEN '2023-04-01 00:00:00' AND '2024-03-31 23:59:59'
),
agg_trip_details AS (
  SELECT
    SUM(trip_duration_seconds) AS total_trip_duration_seconds,
    SUM(passenger_count) AS total_passengers
  FROM
    trip_details
)
SELECT
  'All Zones' AS Zone,
  total_passengers AS annual_passengers,
  CONCAT(
    (total_trip_duration_seconds / 3600)::INT, ':',
    LPAD(((total_trip_duration_seconds % 3600) / 60)::INT::TEXT, 2, '0'), ':',
    LPAD((total_trip_duration_seconds % 60)::INT::TEXT, 2, '0')
  ) AS annual_trip_duration,
  ROUND((total_trip_duration_seconds * 0.001765 / 30), 2) AS annual_expected_tlc_revenue_dollar,
  ROUND(((total_trip_duration_seconds * 0.001765 / 30) * 0.15), 2) AS annual_adv_gross_revenue_dollar
FROM
  agg_trip_details;
''').df()


# 3.2: Calculate the Duration Times Passengers for the Top 5 Zones


In [ ]:
# Calculate the Duration Times Passengers (DTP), tlcTrip revenue & advertising company gross revenue for the Top 5 Zones in the last 3 months.

conn.sql('''
WITH trip_details AS (
  SELECT
    EXTRACT(EPOCH FROM unified_dropoff_datetime) - EXTRACT(EPOCH FROM unified_pickup_datetime) AS trip_duration_seconds,
    passenger_count,
    PUZone
  FROM
    tlctrip
  WHERE unified_pickup_datetime BETWEEN '2024-01-01 00:00:00' AND '2024-03-31 23:59:59'
),
agg_trip_details AS (
  SELECT
    SUM(trip_duration_seconds) AS total_trip_duration_seconds,
    SUM(passenger_count) AS total_passengers,
    PUZone
  FROM
    trip_details
  GROUP BY
    PUZone
),
formatted_trip_duration AS (
  SELECT
    PUZone,
    total_passengers,
    total_trip_duration_seconds,
    (total_trip_duration_seconds / 3600)::INT AS hours,
    ((total_trip_duration_seconds % 3600) / 60)::INT AS minutes,
    (total_trip_duration_seconds % 60)::INT AS seconds
  FROM
    agg_trip_details
),
top_5_zones AS (
  SELECT
    PUZone,
    total_passengers,
    total_trip_duration_seconds,
    (total_trip_duration_seconds / 3600)::INT AS hours,
    ((total_trip_duration_seconds % 3600) / 60)::INT AS minutes,
    (total_trip_duration_seconds % 60)::INT AS seconds
  FROM
    formatted_trip_duration
  ORDER BY
    total_trip_duration_seconds DESC
  LIMIT 5
)
SELECT
  PUZone,
  total_passengers AS passengers,
  CONCAT(hours::TEXT, ':', LPAD(minutes::TEXT, 2, '0'), ':', LPAD(seconds::TEXT, 2, '0')) AS trip_duration,
  ROUND((total_trip_duration_seconds * 0.001765 / 30), 2) AS expected_tlc_revenue_dollar,
  ROUND(((total_trip_duration_seconds * 0.001765 / 30) * 0.15), 2) AS adv_gross_revenue_dollar
FROM
  top_5_zones

UNION ALL

SELECT
  'Total of TOP 5' AS PUZone,
  SUM(total_passengers) AS passengers,
  CONCAT(
    (SUM(total_trip_duration_seconds) / 3600)::INT, ':',
    LPAD(((SUM(total_trip_duration_seconds) % 3600) / 60)::INT::TEXT, 2, '0'), ':',
    LPAD((SUM(total_trip_duration_seconds) % 60)::INT::TEXT, 2, '0')
  ) AS trip_duration,
  ROUND(SUM((total_trip_duration_seconds) * 0.001765 / 30), 2) AS expected_tlc_revenue_dollar,
  ROUND(SUM((total_trip_duration_seconds * 0.001765 / 30) * 0.15), 2) AS adv_gross_revenue_dollar
FROM
  top_5_zones;
''').df()

In [ ]:
# Example SQL query using DuckDB (assuming df32 is your DuckDB result)
query = '''
    WITH trip_details AS (
      SELECT
        EXTRACT(EPOCH FROM unified_dropoff_datetime) - EXTRACT(EPOCH FROM unified_pickup_datetime) AS trip_duration_seconds,
        passenger_count,
        PUZone
      FROM
        tlctrip
      WHERE unified_pickup_datetime BETWEEN '2024-01-01 00:00:00' AND '2024-03-31 23:59:59'
    ),
    agg_trip_details AS (
      SELECT
        SUM(trip_duration_seconds) AS total_trip_duration_seconds,
        SUM(passenger_count) AS total_passengers,
        PUZone
      FROM
        trip_details
      GROUP BY
        PUZone
    ),
    formatted_trip_duration AS (
      SELECT
        PUZone,
        total_passengers,
        total_trip_duration_seconds,
        (total_trip_duration_seconds / 3600)::INT AS hours,
        ((total_trip_duration_seconds % 3600) / 60)::INT AS minutes,
        (total_trip_duration_seconds % 60)::INT AS seconds
      FROM
        agg_trip_details
    ),
    top_5_zones AS (
      SELECT
        PUZone,
        total_passengers,
        total_trip_duration_seconds,
        (total_trip_duration_seconds / 3600)::INT AS hours,
        ((total_trip_duration_seconds % 3600) / 60)::INT AS minutes,
        (total_trip_duration_seconds % 60)::INT AS seconds
      FROM
        formatted_trip_duration
      ORDER BY
        total_trip_duration_seconds DESC
      LIMIT 5
    )
    SELECT
      PUZone,
      total_passengers AS passengers,
      CONCAT(hours::TEXT, ':', LPAD(minutes::TEXT, 2, '0'), ':', LPAD(seconds::TEXT, 2, '0')) AS trip_duration,
      ROUND((total_trip_duration_seconds * 0.001765 / 30), 2) AS expected_tlc_revenue_dollar,
      ROUND(((total_trip_duration_seconds * 0.001765 / 30) * 0.15), 2) AS adv_gross_revenue_dollar
    FROM
      top_5_zones

    UNION ALL

    SELECT
      'Total of TOP 5' AS PUZone,
      SUM(total_passengers) AS passengers,
      CONCAT(
        (SUM(total_trip_duration_seconds) / 3600)::INT, ':',
        LPAD(((SUM(total_trip_duration_seconds) % 3600) / 60)::INT::TEXT, 2, '0'), ':',
        LPAD((SUM(total_trip_duration_seconds) % 60)::INT::TEXT, 2, '0')
      ) AS trip_duration,
      ROUND(SUM((total_trip_duration_seconds) * 0.001765 / 30), 2) AS expected_tlc_revenue_dollar,
      ROUND(SUM((total_trip_duration_seconds * 0.001765 / 30) * 0.15), 2) AS adv_gross_revenue_dollar
    FROM
      top_5_zones;
'''

# Connect to DuckDB and execute the query
df32 = conn.sql(query).df()

# Melt the DataFrame to convert columns into a long format
df32_melted = pd.melt(df32, id_vars=['PUZone'], value_vars=['expected_tlc_revenue_dollar', 'adv_gross_revenue_dollar'],
                      var_name='Revenue Type', value_name='Revenue')

# Customizing the names as per your requirement
df32_melted['Revenue Type'] = df32_melted['Revenue Type'].replace({
    'expected_tlc_revenue_dollar': 'tlctrip_revenue',
    'adv_gross_revenue_dollar': 'advertising_gross_revenue'
})

# Pivot the melted DataFrame so each PUZone has both revenue types as columns
df32_pivot = df32_melted.pivot(index='PUZone', columns='Revenue Type', values='Revenue').reset_index()

# Define colors for each Revenue Type (adjusting blue to be darker)
colors = {'tlctrip_revenue': 'deepskyblue', 'advertising_gross_revenue': 'orange'}

# Plot using Plotly Express
fig = px.bar(df32_pivot, x='PUZone', y=['tlctrip_revenue', 'advertising_gross_revenue'],
             barmode='group',  # Clustered bar mode
             color_discrete_map=colors,
             labels={'PUZone': 'PUZone', 'value': 'Revenue (Dollars)', 'variable': 'Revenue Type'})

fig.show()


# **Q.4**
   * **4.1: Selecting & Sampling Data**
        
   * **4.2: Apply the coefficients to calculat**
   
   * **4.3: OLS Model**
   
   * **4.4: Visualisation**
   
   
   

## 4.1: Selecting & Sampling Data

In [ ]:
# Create 10 percent sample from unique_tlctrip
query = '''
SELECT 
    total_amount,
    trip_distance, 
    passenger_count, 
    RatecodeID, 
    cap_type,  
    year, 
    unified_pickup_datetime,
FROM
    read_parquet('/kaggle/input/final-tlctrip-zone-view/tlctrip3.parquet')
USING SAMPLE 10 PERCENT
'''
smpl = conn.sql(query).df()
smpl
smpl.describe().applymap('{:,.2f}'.format)
smpl.info()

In [ ]:
# List of columns to check for missing values
columns_to_check = ['total_amount', 'trip_distance', 'passenger_count', 'RatecodeID', 'cap_type', 'year', 'unified_pickup_datetime']

# Calculate the number of missing values in each specified column
missing_values = smpl[columns_to_check].isnull().sum()
missing_values

#No missing, so there was no need to drop them

# Drop rows where any of these columns have missing values
# columns_to_clean = ['passenger_count', 'RatecodeID']
# clnd_smpl_10prcnt = smpl_10prcnt.dropna(subset=columns_to_clean)

## 4.2: Apply the coefficients to calculat

In [ ]:
#Build required variables and modify name of the current ones
smpl.rename(columns={'cap_type': 'cab_type'}, inplace=True)
smpl.rename(columns={'trip_distance': 'distance'}, inplace=True)
smpl.rename(columns={'total_amount': 'fare'}, inplace=True)

#Creat 'real_price' which gives prices in 2024
# Convert 'year' column to datetime
smpl['dt_year'] = pd.to_datetime(smpl['year'], format='%Y')
# Create a dictionary of coefficients for each year
coefficients = {
    2020: 1.21,
    2021: 1.16,
    2022: 1.07,
    2023: 1.03,
    2024: 1.00
}

# Apply the coefficients to calculate 'real_price'
smpl['rlfare'] = smpl.apply(lambda row: row['fare'] * coefficients.get(row['dt_year'].year, 1.0), axis=1)
smpl['lnrlfare'] = np.log(smpl['rlfare'])

smpl['hour'] = smpl['unified_pickup_datetime'].dt.hour
smpl['weekday'] = smpl['unified_pickup_datetime'].dt.day_name()
smpl['month'] = smpl['unified_pickup_datetime'].dt.month_name()

smpl['cab_type'] = smpl['cab_type'].astype('category')
smpl['RatecodeID'] = smpl['RatecodeID'].astype('category')
smpl['hour'] = smpl['hour'].astype('category')
smpl['weekday'] = smpl['weekday'].astype('category')
smpl['month'] = smpl['month'].astype('category')
smpl['year'] = smpl['year'].astype('category')

# Remove rows with infinite values if any
smpl = smpl.replace([np.inf, -np.inf], np.nan).dropna()

reg_df = smpl[['lnrlfare', 'distance', 'passenger_count', 'RatecodeID', 'cab_type', 'hour', 'weekday', 'month', 'year']]
reg_df.head(5)
reg_df.info()
reg_df.describe().applymap('{:,.2f}'.format)

## 4.3: OLS Model

In [ ]:
formula = 'lnrlfare ~ distance + passenger_count + C(RatecodeID) + C(cab_type) + C(hour) + C(weekday) + C(month) + C(year)'
model = sm.OLS.from_formula(formula, data=reg_df).fit()
model.summary()

## Notes:
* Standard Errors assume that the covariance matrix of the errors is correctly specified.
* The condition number is large, 5.66e+03. This might indicate that there are strong multicollinearity or other numerical problems.

In [ ]:
# model = sm.OLS.from_formula("lnprice ~ distance + passenger_count + C(RatecodeID) + C(cab_type) + C(hour) + C(weekday) + C(month) + C(year)", data=reg_df).fit()
formula = 'lnrlfare ~ distance + passenger_count + C(RatecodeID) + C(cab_type) + C(hour) + C(weekday) + C(month) + C(year)'
model = sm.OLS.from_formula(formula, data=reg_df).fit()
model.summary()

In [ ]:
# Extract coefficients of the regression
coef_df = model.params.reset_index()
coef_df.columns = ['variable', 'coefficient']
coef_df
coef_df.info()

# Extract statistics
stderr_df = model.bse.reset_index()
stderr_df.columns = ['variable', 'std_err']

tvalues_df = model.tvalues.reset_index()
tvalues_df.columns = ['variable', 'tvalue']

pvalues_df = model.pvalues.reset_index()
pvalues_df.columns = ['variable', 'pvalue']

conf_int_df = model.conf_int().reset_index()
conf_int_df.columns = ['variable', 'ci_lower', 'ci_upper']

# Merge these statistics into coef_df
coef_df = coef_df.merge(stderr_df, on='variable')
coef_df = coef_df.merge(tvalues_df, on='variable')
coef_df = coef_df.merge(pvalues_df, on='variable')
coef_df = coef_df.merge(conf_int_df, on='variable')

coef_df
coef_df.to_csv('regression results.csv', index=False)

In [ ]:
# Filter for 'C(hour)' variables
hour_coef_df = coef_df[coef_df['variable'].str.contains(r'C\(hour\)\[T\.\d+\]')]
# Extract hour values from the variable names
hour_coef_df['hour'] = hour_coef_df['variable'].str.extract(r'C\(hour\)\[T\.(\d+)\]', expand=False).astype(int)

hour_coef_df

## Visualisation

In [ ]:
# Create the plot
fig_hour = go.Figure()

# Add the main line for coefficients
fig_hour.add_trace(go.Scatter(
    x=hour_coef_df['hour'],
    y=hour_coef_df['coefficient'],
    mode='lines+markers',
    name='Coefficient'
))

# Add the confidence interval shaded area
fig_hour.add_trace(go.Scatter(
    x=pd.concat([hour_coef_df['hour'], hour_coef_df['hour'][::-1]]),
    y=pd.concat([hour_coef_df['ci_upper'], hour_coef_df['ci_lower'][::-1]]),
    fill='toself',
    fillcolor='rgba(0,100,80,0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    showlegend=True,
    name='95% CI'
))

# Add a horizontal line at zero
fig_hour.add_shape(
    type='line',
    x0=0,
    y0=0,
    x1=1,
    y1=0,
    xref='paper',
    yref='y',
    line=dict(color='darkgray', width=2)
)

# Customize the layout
fig_hour.update_layout(
    title='Estimated Coefficients for C(hour) with 95% Confidence Interval',
    xaxis_title='Hour',
    yaxis_title='Coefficient',
    template='plotly_white',
    # Customize the layout to show all hour values on the x-axis
    xaxis=dict(
        tickmode='array',
        tickvals=list(range(24)),
        ticktext=[str(hour) for hour in range(24)]
    )
)

In [ ]:
# Filter for 'C(weekday)' variables
wd_coef_df = coef_df[coef_df['variable'].str.contains(r'C\(weekday\)\[T\.\w+\]')]
wd_coef_df
# Extract weekdays from the variable names
wd_coef_df['weekday'] = wd_coef_df['variable'].str.extract(r'C\(weekday\)\[T\.(\w+)\]', expand=False)

# Sort the dataframes by 'weekday' for proper plotting
# Mapping of day names to day numbers
day_mapping = {
    'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5, 'Saturday': 6,
    'Sunday': 7
}

# Map month names to month numbers
wd_coef_df['day_number'] = wd_coef_df['weekday'].map(day_mapping)

# Sort by month number
wd_coef_df = wd_coef_df.sort_values('day_number')
wd_coef_df

In [ ]:
# Create the plot for 'month' coefficients
fig_wd = go.Figure()

# Add the main line for 'month' coefficients
fig_wd.add_trace(go.Scatter(
    x=wd_coef_df['weekday'],
    y=wd_coef_df['coefficient'],
    mode='lines+markers',
    name='Coefficient'
))

# Add error bars as a separate trace
fig_wd.add_trace(go.Scatter(
    x=wd_coef_df['weekday'],
    y=wd_coef_df['coefficient'],
    mode='markers',
    name='95% CI',
    error_y=dict(
        type='data',
        symmetric=False,
        array=wd_coef_df['ci_upper'] - wd_coef_df['coefficient'],
        arrayminus=wd_coef_df['coefficient'] - wd_coef_df['ci_lower'],
        width=1.5,
        thickness=1.5,
        color='rgba(0,100,80,0.2)'
    ),
    marker=dict(color='rgba(0,100,80,0.2)')
))

# Add a horizontal line at zero
fig_wd.add_shape(
    type='line',
    x0=0,
    y0=0,
    x1=1,
    y1=0,
    xref='paper',
    yref='y',
    line=dict(color='darkgray', width=2)
)

# Customize the layout
fig_wd.update_layout(
    title='Estimated Coefficients for C(weekday) with 95% Confidence Intervals',
    xaxis_title='Weekdays',
    yaxis_title='Coefficient',
    template='plotly_white',
    xaxis=dict(
        tickmode='array',
        tickvals=wd_coef_df['weekday'],
        ticktext=wd_coef_df['weekday']
    )
)


In [ ]:
# Create the plot for 'month' coefficients
fig_wd = go.Figure()

# Add the main line for 'month' coefficients
fig_wd.add_trace(go.Scatter(
    x=wd_coef_df['weekday'],
    y=wd_coef_df['coefficient'],
    mode='lines+markers',
    name='Coefficient'
))

# Add error bars as a separate trace
fig_wd.add_trace(go.Scatter(
    x=wd_coef_df['weekday'],
    y=wd_coef_df['coefficient'],
    mode='markers',
    name='95% CI',
    error_y=dict(
        type='data',
        symmetric=False,
        array=wd_coef_df['ci_upper'] - wd_coef_df['coefficient'],
        arrayminus=wd_coef_df['coefficient'] - wd_coef_df['ci_lower'],
        width=1.5,
        thickness=1.5,
        color='rgba(0,100,80,0.2)'
    ),
    marker=dict(color='rgba(0,100,80,0.2)')
))

# Add a horizontal line at zero
fig_wd.add_shape(
    type='line',
    x0=0,
    y0=0,
    x1=1,
    y1=0,
    xref='paper',
    yref='y',
    line=dict(color='darkgray', width=2)
)

# Customize the layout
fig_wd.update_layout(
    title='Estimated Coefficients for C(weekday) with 95% Confidence Intervals',
    xaxis_title='Weekdays',
    yaxis_title='Coefficient',
    template='plotly_white',
    xaxis=dict(
        tickmode='array',
        tickvals=wd_coef_df['weekday'],
        ticktext=wd_coef_df['weekday']
    )
)


In [ ]:
# Filter for 'C(month)' variables
month_coef_df = coef_df[coef_df['variable'].str.contains(r'C\(month\)\[T\.\w+\]')]
# Extract month values from the variable names
month_coef_df['month'] = month_coef_df['variable'].str.extract(r'C\(month\)\[T\.(\w+)\]', expand=False)
month_coef_df

# Sort the dataframes by 'month' for proper plotting
# Mapping of month names to month numbers
month_mapping = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5, 'June': 6,
    'July': 7, 'August': 8, 'September': 9, 'October': 10, 'November': 11, 'December': 12
}

# Map month names to month numbers
month_coef_df['month_number'] = month_coef_df['month'].map(month_mapping)

# Sort by month number
month_coef_df = month_coef_df.sort_values('month_number')

month_coef_df

In [ ]:
# Create the plot for 'month' coefficients
fig_month = go.Figure()

# Add the main line for 'month' coefficients
fig_month.add_trace(go.Scatter(
    x=month_coef_df['month'],
    y=month_coef_df['coefficient'],
    mode='lines+markers',
    name='Coefficient'
))

# Add error bars as a separate trace
fig_month.add_trace(go.Scatter(
    x=month_coef_df['month'],
    y=month_coef_df['coefficient'],
    mode='markers',
    name='95% CI',
    error_y=dict(
        type='data',
        symmetric=False,
        array=month_coef_df['ci_upper'] - month_coef_df['coefficient'],
        arrayminus=month_coef_df['coefficient'] - month_coef_df['ci_lower'],
        width=1.5,
        thickness=1.5,
        color='rgba(0,100,80,0.2)'
    ),
    marker=dict(color='rgba(0,100,80,0.2)')
))

# Add a horizontal line at zero
fig_month.add_shape(
    type='line',
    x0=0,
    y0=0,
    x1=1,
    y1=0,
    xref='paper',
    yref='y',
    line=dict(color='darkgray', width=2)
)

# Customize the layout
fig_month.update_layout(
    title='Estimated Coefficients for C(month) with 95% Confidence Intervals',
    xaxis_title='Months',
    yaxis_title='Coefficient',
    template='plotly_white',
    xaxis=dict(
        tickmode='array',
        tickvals=month_coef_df['month'],
        ticktext=month_coef_df['month']
    )
)


In [ ]:
# Filter for 'C(year)' variables
yr_coef_df = coef_df[coef_df['variable'].str.contains(r'C\(year\)\[T\.\d+\]')]

# Extract hour values from the variable names
yr_coef_df['year'] = yr_coef_df['variable'].str.extract(r'C\(year\)\[T\.(\d+)\]', expand=False).astype(int)
yr_coef_df

In [ ]:
# Create the plot for 'year' coefficients
fig_yr = go.Figure()

# Add the main line for 'year' coefficients
fig_yr.add_trace(go.Scatter(
    x=yr_coef_df['year'],
    y=yr_coef_df['coefficient'],
    mode='lines+markers',
    name='Coefficient'
))

# Add error bars as a separate trace
fig_yr.add_trace(go.Scatter(
    x=yr_coef_df['year'],
    y=yr_coef_df['coefficient'],
    mode='markers',
    name='95% CI',
    error_y=dict(
        type='data',
        symmetric=False,
        array=yr_coef_df['ci_upper'] - yr_coef_df['coefficient'],
        arrayminus=yr_coef_df['coefficient'] - yr_coef_df['ci_lower'],
        width=1.5,
        thickness=1.5,
        color='rgba(0,100,80,0.2)'
    ),
    marker=dict(color='rgba(0,100,80,0.2)')
))

# Add a horizontal line at zero
fig_yr.add_shape(
    type='line',
    x0=0,
    y0=0,
    x1=1,
    y1=0,
    xref='paper',
    yref='y',
    line=dict(color='darkgray', width=2)
)

# Customize the layout
fig_yr.update_layout(
    title='Estimated Coefficients for C(year) with 95% Confidence Intervals',
    xaxis_title='Years',
    yaxis_title='Coefficient',
    template='plotly_white',
    xaxis=dict(
        tickmode='array',
        tickvals=yr_coef_df['year'],
        ticktext=yr_coef_df['year']
    )
)


# **Q.5**
   * **5.1: Suggested locations for the deployment of drivers**
        
   * **5.2: Locations we Start**
   
   * **5.3: Calculation of the busiest destinations for the proposed destinations**
        * 5.3.1: The busiest routes at 00:00 to 6:00 in the morning
        * 5.3.2: The busiest routes at 06:00 to 12:00 in the morning
        * 5.3.3: The busiest routes at 12:00 to 18:00 in the morning
        * 5.3.4: The busiest routes at 18:00 to 23:59 in the morning

   * **5.4: Graph of the number of repetitions of each busy route in the 4 desired time periods**
   
   * **5.5: Graph of the total number of trips in the total hours of the day for each route**
 
   
   

## 5.1: Suggested locations for the deployment of drivers

In [ ]:
conn.sql('''
CREATE OR REPLACE VIEW new_tlctrip AS
SELECT 
     PUZone as start_zone
    ,PUlocationID AS start_location_id 
    ,count(*) AS number_of_rides
    ,sum(total_amount) AS sum_of_price
    ,percentile_disc(0.50) WITHIN GROUP (ORDER BY total_amount) AS price_quartile_50
    ,number_of_rides * price_quartile_50 AS metric
    ,sum_of_price/(SELECT SUM(total_amount) FROM tlctrip WHERE unified_pickup_datetime BETWEEN '2023-04-01 00:00:00' AND '2024-03-31 00:00:00')*100 AS market_cap_percentage
FROM tlctrip
WHERE 
    unified_pickup_datetime BETWEEN '2023-04-01 00:00:00' AND '2024-03-31 00:00:00'  
GROUP BY 1,2
ORDER BY 6 desc
''')

## 5.2: Location we start

We have started top 5 locations which have 30 percent of trips

In [ ]:
df51 = conn.sql('''
SELECT 
    *
    ,SUM(market_cap_percentage) OVER (ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cumulative_market_cap_percentage
FROM new_tlctrip
LIMIT 5
''').to_df()
df51

## 5.3: Calculation of the busiest destinations for the proposed destinations

In [ ]:
df52 = conn.sql('''
WITH ranked_rides AS (
  SELECT 
    PUZone AS start_location,
    DOZone AS end_location,
    COUNT(*) AS number_of_rides,
    DENSE_RANK() OVER (PARTITION BY start_location ORDER BY number_of_rides DESC) AS rank
  FROM 
    tlctrip
  WHERE
    unified_pickup_datetime BETWEEN '2023-04-01 00:00:00' AND '2024-03-31 00:00:00' 
    AND start_location in ('JFK Airport','LaGuardia Airport','Midtown Center','Upper East Side South', 'Upper East Side North')
    GROUP BY 1,2
)
SELECT *
FROM ranked_rides
WHERE rank=1
ORDER BY start_location;''').to_df()
df52

### 5.3.1: The busiest routes at 00:00 to 6:00 in the morning

In [ ]:
df52 = conn.sql('''
WITH ranked_rides AS (
  SELECT 
    PUZone AS start_location,
    DOZone AS end_location,
    COUNT(*) AS number_of_rides,
    DENSE_RANK() OVER (PARTITION BY start_location ORDER BY number_of_rides DESC) AS rank
  FROM 
    tlctrip
  WHERE
    unified_pickup_datetime BETWEEN '2023-04-01 00:00:00' AND '2024-03-31 00:00:00' 
    AND start_location in ('JFK Airport','LaGuardia Airport','Midtown Center','Upper East Side South', 'Upper East Side North')
    AND HOUR(unified_pickup_datetime) between 0 and 5
    GROUP BY 1,2
)
SELECT *
FROM ranked_rides
WHERE rank=1
ORDER BY start_location;''').to_df()
df52

### 5.3.2: The busiest routes at 06:00 to 12:00 in the morning

In [ ]:
df53 = conn.sql('''
WITH ranked_rides AS (
  SELECT 
    PUZone AS start_location,
    DOZone AS end_location,
    COUNT(*) AS number_of_rides,
    DENSE_RANK() OVER (PARTITION BY start_location ORDER BY number_of_rides DESC) AS rank
  FROM 
    tlctrip
  WHERE
    unified_pickup_datetime BETWEEN '2023-04-01 00:00:00' AND '2024-03-31 00:00:00' 
    AND start_location in ('JFK Airport','LaGuardia Airport','Midtown Center','Upper East Side South', 'Upper East Side North')
    AND HOUR(unified_pickup_datetime) between 6 and 11
    GROUP BY 1,2
)
SELECT *
FROM ranked_rides
WHERE rank=1
ORDER BY start_location;''').to_df()
df53

## 5.3.3: The busiest routes at 12:00 to 18:00 in the morning

In [ ]:
df54 = conn.sql('''
WITH ranked_rides AS (
  SELECT 
    PUZone AS start_location,
    DOZone AS end_location,
    COUNT(*) AS number_of_rides,
    DENSE_RANK() OVER (PARTITION BY start_location ORDER BY number_of_rides DESC) AS rank
  FROM 
    tlctrip
  WHERE
    unified_pickup_datetime BETWEEN '2023-04-01 00:00:00' AND '2024-03-31 00:00:00' 
    AND start_location in ('JFK Airport','LaGuardia Airport','Midtown Center','Upper East Side South', 'Upper East Side North')
    AND HOUR(unified_pickup_datetime) between 12 and 17
    GROUP BY 1,2
)
SELECT *
FROM ranked_rides
WHERE rank=1
ORDER BY start_location;''').to_df()
df54

### 5.3.4: The busiest routes at 18:00 to 23:59 in the morning

In [ ]:
df55 = conn.sql('''
WITH ranked_rides AS (
  SELECT 
    PUZone AS start_location,
    DOZone AS end_location,
    COUNT(*) AS number_of_rides,
    DENSE_RANK() OVER (PARTITION BY start_location ORDER BY number_of_rides DESC) AS rank
  FROM 
    tlctrip
  WHERE
    unified_pickup_datetime BETWEEN '2023-04-01 00:00:00' AND '2024-03-31 00:00:00' 
    AND start_location in ('JFK Airport','LaGuardia Airport','Midtown Center','Upper East Side South', 'Upper East Side North')
    AND HOUR(unified_pickup_datetime) between 18 and 23
    GROUP BY 1,2
)
SELECT *
FROM ranked_rides
WHERE rank=1
ORDER BY start_location;''').to_df()
df55

In [ ]:
# Create a list containing all your DataFrames
df_list = [df52, df53, df54, df55]

# Concatenate the DataFrames vertically (adding rows)
df_combined = pd.concat(df_list)

# Print the result
df_combined

In [ ]:
df56 = conn.sql('''
SELECT 
     start_location
    ,end_location
    ,COUNT(*) AS number_of_occurance
    ,SUM(number_of_rides) AS Total_number_of_rides
FROM df_combined
GROUP BY 1,2
''').to_df()

df56['path'] = df56['start_location'] + ' - ' + df56['end_location']
df56

## 5.4: Graph of the number of repetitions of each busy route in the 4 desired time periods

In [ ]:
df_sorted = df56.sort_values(by='number_of_occurance', ascending=True)

# Create the horizontal bar chart using px.bar
fig = px.bar(
    df_sorted,  # Data source (sorted DataFrame)
    x='number_of_occurance',  # x-axis (market_cap_percentage)
    y='path',  # y-axis (zone)
    title='Occurance of Path in different time periods',
    orientation='h',  # Horizontal orientation
    #color='path',  # Color based on market_cap_percentage
    text=df_sorted['number_of_occurance'].round(1),  # Display values as text on bars
    labels={  # Customize axis labels
        'path': 'Path Name',
        'number_of_occurance': 'Occurance'
    }
)

# Display the chart
fig.show(renderer='iframe')

## 5.5: Graph of the total number of trips in the total hours of the day for each route

In [ ]:
df_sorted = df56.sort_values(by='Total_number_of_rides', ascending=True)

# Create the horizontal bar chart using px.bar
fig = px.bar(
    df_sorted,  # Data source (sorted DataFrame)
    x='Total_number_of_rides',  # x-axis (market_cap_percentage)
    y='path',  # y-axis (zone)
    title='Total Number of rides in 1 Recent year For Each Path ',
    orientation='h',  # Horizontal orientation
    #color='path',  # Color based on market_cap_percentage
    text=df_sorted['Total_number_of_rides'].round(1),  # Display values as text on bars
    labels={  # Customize axis labels
        'path': 'Path Name',
        'Total_number_of_rides':'Number Of Rides '
    }
)

# Display the chart
fig.show(renderer='iframe')

# **Q.6**
   * **6.1: Creating View and Dataframe For Analyzing Data**
        
   * **6.2: Calculating Correlation and Statistics**
   
   * **6.3: Visualisation**
   

 
 

## 6.1: Creating View and Dataframe For Analyzing Data

In [ ]:
conn.sql('''
CREATE OR REPLACE VIEW num6_prepration AS
SELECT
    unified_pickup_datetime AS Pickup_Datetime,
    trip_distance * 1.60934 AS Distance_KM,
    total_amount AS Price,
    (EXTRACT(EPOCH FROM unified_dropoff_datetime) - EXTRACT(EPOCH FROM unified_pickup_datetime))/60 AS Duration_Min,
    HOUR(unified_pickup_datetime) AS Time_Hour,
    CASE
        WHEN Time_Hour BETWEEN 0 AND 3 THEN '1'
        WHEN Time_Hour BETWEEN 4 AND 7 THEN '2'
        WHEN Time_Hour BETWEEN 8 AND 11 THEN '3'
        WHEN Time_Hour BETWEEN 12 AND 15 THEN '4'
        WHEN Time_Hour BETWEEN 16 AND 19 THEN '5'
        WHEN Time_Hour BETWEEN 20 AND 23 THEN '6'
        ELSE '7'
    END AS Time_Bins,
    CASE
    WHEN trip_distance * 1.60934 < 2 THEN 1
    WHEN trip_distance * 1.60934 < 4 THEN 2
    WHEN trip_distance * 1.60934 < 6 THEN 3
    WHEN trip_distance * 1.60934 < 8 THEN 4
    WHEN trip_distance * 1.60934 < 10 THEN 5
    WHEN trip_distance * 1.60934 < 12 THEN 6
    WHEN trip_distance * 1.60934 < 14 THEN 7
    WHEN trip_distance * 1.60934 < 16 THEN 8
    WHEN trip_distance * 1.60934 < 18 THEN 9
    WHEN trip_distance * 1.60934 < 20 THEN 10
    WHEN trip_distance * 1.60934 < 22 THEN 11
    WHEN trip_distance * 1.60934 < 24 THEN 12
    WHEN trip_distance * 1.60934 < 26 THEN 13
    WHEN trip_distance * 1.60934 < 28 THEN 14
    WHEN trip_distance * 1.60934 < 30 THEN 15
    WHEN trip_distance * 1.60934 < 32 THEN 16
    WHEN trip_distance * 1.60934 < 34 THEN 17
    WHEN trip_distance * 1.60934 < 36 THEN 18
    WHEN trip_distance * 1.60934 < 38 THEN 19
    WHEN trip_distance * 1.60934 < 40 THEN 20
    WHEN trip_distance * 1.60934 < 42 THEN 21
    WHEN trip_distance * 1.60934 < 44 THEN 22
    WHEN trip_distance * 1.60934 < 46 THEN 23
    WHEN trip_distance * 1.60934 < 48 THEN 24
    WHEN trip_distance * 1.60934 < 50 THEN 25
    ELSE 26
    END AS Distance_Bin,
    (trip_distance * 1.60934) / ((EXTRACT(EPOCH FROM unified_dropoff_datetime) - EXTRACT(EPOCH FROM unified_pickup_datetime))/3600) AS Speed,
    CASE
    WHEN (trip_distance * 1.60934) / ((EXTRACT(EPOCH FROM unified_dropoff_datetime) - EXTRACT(EPOCH FROM unified_pickup_datetime))/3600) < 10 THEN 1
    WHEN (trip_distance * 1.60934) / ((EXTRACT(EPOCH FROM unified_dropoff_datetime) - EXTRACT(EPOCH FROM unified_pickup_datetime))/3600) < 20 THEN 2
    WHEN (trip_distance * 1.60934) / ((EXTRACT(EPOCH FROM unified_dropoff_datetime) - EXTRACT(EPOCH FROM unified_pickup_datetime))/3600) < 30 THEN 3
    WHEN (trip_distance * 1.60934) / ((EXTRACT(EPOCH FROM unified_dropoff_datetime) - EXTRACT(EPOCH FROM unified_pickup_datetime))/3600) < 40 THEN 4
    WHEN (trip_distance * 1.60934) / ((EXTRACT(EPOCH FROM unified_dropoff_datetime) - EXTRACT(EPOCH FROM unified_pickup_datetime))/3600) < 50 THEN 5
    WHEN (trip_distance * 1.60934) / ((EXTRACT(EPOCH FROM unified_dropoff_datetime) - EXTRACT(EPOCH FROM unified_pickup_datetime))/3600) < 60 THEN 6
    WHEN (trip_distance * 1.60934) / ((EXTRACT(EPOCH FROM unified_dropoff_datetime) - EXTRACT(EPOCH FROM unified_pickup_datetime))/3600) < 70 THEN 7
    ELSE 8
    END AS Speed_Bin
FROM
    tlctrip
WHERE
Pickup_Datetime > '2023-03-01';
''')


In [ ]:
conn.sql('''CREATE OR REPLACE VIEW df AS
SELECT 
    Pickup_Datetime,
    Distance_KM,
    Time_Hour,
    Speed,
    Speed_Bin,
    CAST(Time_Bins AS INT) AS Time_Bin,
    CAST(Speed_Bin AS INT) AS Speed_Bin,
    CAST(Distance_Bin AS INT) AS Distance_Bin,
    Price,
    EXTRACT(EPOCH FROM Pickup_Datetime) AS Pickup_Datetime_Epoch,
FROM num6_prepration''')
# Analyze correlation and statistical results

df51 = conn.sql('''SELECT 
    Pickup_Datetime,
    Distance_Bin,
    Speed_Bin,
    Time_Bin,
    Price,
    Pickup_Datetime_Epoch,
FROM df''').to_df()


## 6.2: Calculating Correlation and Statistics

In [ ]:
correlation_results = []

for ( dist_bin, time_bin, speed_bin), group in df51.groupby(['Distance_Bin', 'Time_Bin', 'Speed_Bin']):
    correlation = group['Pickup_Datetime_Epoch'].corr(group['Price'])
    r_squared = correlation ** 2  # Calculate the coefficient of determination
    correlation_results.append({
        'Distance_Bin': dist_bin,
        'Time_Bin': time_bin,
        'Speed_Bin': speed_bin,
        'Correlation': correlation,
        'Coefficient': r_squared  # Add R_squared value to the results
    })

# Create a DataFrame from the correlation results
correlation_df = pd.DataFrame(correlation_results)


In [ ]:
conn.sql('''CREATE OR REPLACE VIEW statistics AS
SELECT
    Distance_Bin,
    Time_Bin,
    Speed_Bin,
    Count (*) AS Count,
    AVG(Price) AS Mean_Price,
    MEDIAN(Price) AS Median_Price,
    VAR_SAMP(Price) AS Variance_Price,
    STDDEV(Price) AS Stddev_Price,
    MIN(Price) AS Min_Price,
    PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY Price) AS Q1_Price,
    PERCENTILE_CONT(0.50) WITHIN GROUP (ORDER BY Price) AS Q2_Price,
    PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY Price) AS Q3_Price,
    MAX(Price) AS Max_Price,
FROM
    df
GROUP BY
    Distance_Bin,
    Time_Bin,
    Speed_Bin,
HAVING
    COUNT(*) >= 20 
ORDER BY
    Distance_Bin,
    Time_Bin,
    Speed_Bin;
    ''')
statistics = conn.sql('''select * from statistics ''').to_df()

In [ ]:
# Create a unique identifier for each combination of , Distance_Bin, Time_Bins, and Speed_Bin in the statistics DataFrame
statistics['unique_id'] = statistics['Distance_Bin'].astype(str) + '_' + \
                          statistics['Time_Bin'].astype(str) + '_' + \
                          statistics['Speed_Bin'].astype(str)

# Create a unique identifier for each combination of PULocationID, Distance_Bin, Time_Bins, and Speed_Bin in the correlation_df DataFrame
correlation_df['unique_id'] = correlation_df['Distance_Bin'].astype(str) + '_' + \
                              correlation_df['Time_Bin'].astype(str) + '_' + \
                              correlation_df['Speed_Bin'].astype(str)

# Merge the dataframes based on the unique identifier
merged_df = pd.merge(correlation_df, statistics, on='unique_id', how='inner')

# Drop the 'unique_id' column if not needed
merged_df.drop(columns=['unique_id'], inplace=True)

# Now merged_df contains the combined information from both dataframes
merged_df.head()


## 6.3: Visualisation

In [ ]:
columns_to_plot = ['Correlation', 'Coefficient', 'Count', 
                   'Mean_Price', 'Median_Price', 
                   'Variance_Price', 'Stddev_Price']

# Determine the number of rows and columns for the subplots (here, 4 rows and 2 columns are used)
fig = make_subplots(rows=4, cols=2, 
                    subplot_titles=columns_to_plot,
                    shared_yaxes=False)

# Loop to add each histogram to its respective subplot position
for i, column in enumerate(columns_to_plot, 1):
    row = (i - 1) // 2 + 1  # Calculate the row number for the subplot
    col = (i - 1) % 2 + 1   # Calculate the column number for the subplot
    fig.add_trace(
        go.Histogram(x=merged_df[column], name=f'Distribution of {column}'),
        row=row, col=col
    )

# Update the visual layout settings of the figure
fig.update_layout(
    title_text="Distribution of Various Columns",
    height=1200,  # Set the height as desired
    width=1000,   # Set the width as desired
    showlegend=False  # Do not show the legend box
)

# Show the figure
fig.show()


In [ ]:
# Assume df51 is already defined and contains the necessary data

# Mapping dictionaries for labels
time_bin_labels = {
    1: '0-4 AM', 2: '4-8 AM', 3: '8-12 PM',
    4: '12-4 PM', 5: '4-8 PM', 6: '8-12 AM'
}

speed_bin_labels = {
    1: '0 to 10 KM/H', 2: '10 to 20 KM/H', 3: '20 to 30 KM/H', 4: '30 to 40 KM/H', 
    5: '40 to 50 KM/H', 6: '50 to 60 KM/H', 7: '60 to 70 KM/H', 8: 'More than 70 KM/H'
}

distance_bin_labels = {
    1: '2 KM', 2: '4 KM', 3: '6 KM', 4: '8 KM',
    5: '10 KM', 6: '12 KM', 7: '14 KM', 8: '16 KM',
    9: '18 KM', 10: '20 KM', 11: '22 KM', 12: '24 KM',
    13: '26 KM', 14: '28 KM', 15: '30 KM', 16: '32 KM',
    17: '34 KM', 18: '36 KM', 19: '38 KM', 20: '40 KM',
    21: '42 KM', 22: '44 KM', 23: '46 KM', 24: '48 KM',
    25: '50 KM', 26: 'More than 50 KM'
}

speed_bin_range = range(1, 9)  # Speed_Bin range: 1 to 8
distance_bin_range = range(1, 27)  # Distance_Bin range: 1 to 26
time_bin_range = range(1, 7)  # Time_Bin range: 1 to 6

# Iterate through all combinations of Distance_Bin, Time_Bin, and Speed_Bin
for distance_bin in distance_bin_range:
    for time_bin in time_bin_range:
        for speed_bin in speed_bin_range:
            # Filter the data for the current combination
            filtered_data = df51[
                (df51['Distance_Bin'] == distance_bin) &
                (df51['Time_Bin'] == time_bin) &
                (df51['Speed_Bin'] == speed_bin)
            ].copy()

            # Check if there is data for this combination, if not, skip to the next
            if filtered_data.empty:
                continue

            # Sort the data by Pickup_Datetime
            filtered_data.sort_values(by='Pickup_Datetime', inplace=True)

            # Calculate the count of the filtered data
            count = len(filtered_data)

            # Create the scatter plot with trendline using Plotly Express
            fig = px.scatter(filtered_data, x='Pickup_Datetime', y='Price',
                             title=f'Price Change Over Time - Distance: {distance_bin_labels[distance_bin]}, '
                                   f'Time: {time_bin_labels[time_bin]}, Speed: {speed_bin_labels[speed_bin]} - Count: {count}',
                             labels={'Pickup_Datetime': 'Pickup Datetime', 'Price': 'Price'},
                             trendline="ols")  # Adding trendline with method="ols" (ordinary least squares)

            # Show the plot
            fig.show()

# **Q.7**
   * **7.1: Creating View and Dataframe For Analyzing Data**
        
   * **7.2: Finding Same Path**
   
   * **7.3: Save the result_df to a CSV file**
   

 
 

## 7.1: Creating View and Dataframe For Analyzing Data



In [ ]:
# Create a view table of our dataset.


conn.sql('''
CREATE OR REPLACE VIEW tlctrip AS
SELECT 
    *,
    CASE 
        WHEN cap_type = 'green' THEN lpep_pickup_datetime
        WHEN cap_type = 'yellow' THEN tpep_pickup_datetime
        ELSE NULL
    END AS unified_pickup_datetime,
    CASE 
        WHEN cap_type = 'green' THEN lpep_dropoff_datetime
        WHEN cap_type = 'yellow' THEN tpep_dropoff_datetime
        ELSE NULL
    END AS unified_dropoff_datetime
FROM 
    read_parquet('/kaggle/input/tlc-trip-2020-01-to-2024-03/trips/*/*/*.parquet', hive_partitioning = true)
WHERE
    (
        unified_pickup_datetime < '2024-04-01' AND 
        unified_pickup_datetime >= '2020-01-01'
    ) AND
    (
        (fare_amount >= 0) AND
        (total_amount >= 0) AND
        ((extra >= 0) OR (extra IS NULL)) AND
        ((mta_tax >= 0) OR (mta_tax IS NULL)) AND
        ((tip_amount >= 0) OR (tip_amount IS NULL)) AND
        ((tolls_amount >= 0) OR (tolls_amount IS NULL)) AND
        ((ehail_fee >= 0) OR (ehail_fee IS NULL)) AND
        ((improvement_surcharge >= 0) OR (improvement_surcharge IS NULL)) AND
        ((congestion_surcharge >= 0) OR (congestion_surcharge IS NULL)) AND
        ((airport_fee >= 0) OR (airport_fee IS NULL))
    )
''')

# Create a view table of NewYork taxizone dataset.

conn.sql('''
CREATE OR REPLACE VIEW taxizone AS 
SELECT * FROM 'https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv'
''')

# Get a view of our tlctrip view.

conn.sql('''
SELECT * FROM tlctrip LIMIT 2
''').df()

# Get a view of our taxizone view.

conn.sql('''
SELECT * FROM taxizone LIMIT 2
''').df()

# Get a look on fields data types.

conn.sql('''
DESCRIBE tlctrip
''').df()

In [ ]:
conn.sql('''
-- Get only the column names for the taxizone view
SELECT column_name
FROM information_schema.columns
WHERE table_name = 'taxizone';

''')

## 7.2: Finding Same Paths

In [ ]:
# SQL query to count trips with the same start and end locations for the year 2023
query = '''
SELECT 
    PULocationID, 
    DOLocationID, 
    COUNT(*) AS trip_count
FROM 
    tlctrip
WHERE
    unified_pickup_datetime BETWEEN '2023-04-01' AND '2024-03-31'
GROUP BY 
    PULocationID, 
    DOLocationID
ORDER BY 
    trip_count DESC
'''

# Execute the query and fetch the result into a Pandas DataFrame
SamePath = conn.sql(query).df()

SamePath

In [ ]:
# Step 1: Extract the hour of day and count trips
query = '''
SELECT 
    PUZone, 
    DOZone,
    EXTRACT(HOUR FROM unified_pickup_datetime) AS hour_of_day,
    COUNT(*) AS trip_count
FROM 
    tlctrip
WHERE
    unified_pickup_datetime BETWEEN '2023-04-01' AND '2024-03-31'
GROUP BY 
    PUZone, 
    DOZone,
    hour_of_day
ORDER BY 
    PUZone, 
    DOZone,
    hour_of_day
'''

# Execute the query and fetch the result into a Pandas DataFrame
df_hourly_distribution = conn.sql(query).df()

# Step 2: Create the pivot table
pivot_df = df_hourly_distribution.pivot_table(
    index=['PUZone', 'DOZone'], 
    columns='hour_of_day', 
    values='trip_count', 
    fill_value=0
)

# Step 3: Calculate the average trip_count for each hour
avg_trip_count_per_hour = pivot_df.mean()

# Step 4: Filter rows where all trip_counts are above average for each hour
above_average_paths = pivot_df.apply(lambda row: all(row > avg_trip_count_per_hour[row.index]), axis=1)

filtered_df = pivot_df[above_average_paths]

# Step 5: Calculate the total trip_count for each row (PULocationID, DOLocationID)
filtered_df['total_trip_count'] = filtered_df.sum(axis=1)

# Step 6: Get the top 5 rows based on total_trip_count
top_5_rows = filtered_df.sort_values(by='total_trip_count', ascending=False).head(10)

# Display the top 5 rows
top_5_rows.reset_index()

## 7.3: Save the result_df to a CSV file

In [ ]:
# Save the result_df to a CSV file
top_5_rows.to_csv('path_distribution.csv', index=False)

# Optionally, display the path to the saved file
print('CSV file saved as path_distribution.csv')

# **Q.8**
        
   * **8.1: Finding Top 1%**
   
   * **8.2: Distribution of Counts by Hour**
   
   * **8.3: Distribution of Counts by Weekdays**
   
   * **8.4: Distribution of Counts by Weather Condition**
   
   * **8.5: Calculate Correlations**
   

 
 

## 8.1: Finding Top 1% 

In [ ]:
# Fetch all results
TOP = conn.sql('''
select 
    year(unified_pickup_datetime) as year,
    month(unified_pickup_datetime) as month,
    day(unified_pickup_datetime) as day,
    hour(unified_pickup_datetime) as hour,
    count(*) as count
from tlctrip
group by 1, 2, 3, 4
order by count desc
''').to_df()

print ("length TOP is : ",len(TOP))
TOP1=TOP.head(372)
print ("length TOP1 is : ",len(TOP1))
TOP1


## 8.2: Distribution of Counts by Hour

In [ ]:
fig = px.histogram(TOP1, x='hour', nbins=12, title='Distribution of Counts by Hour', text_auto=True)

# Update layout to show the data labels more clearly
fig.update_traces(textposition='outside', textfont_size=12)
fig.show(renderer='iframe')

In [ ]:
TOP1['date'] = pd.to_datetime(TOP1[['year', 'month', 'day']])

# Extract weekday names
TOP1['weekday'] = TOP1['date'].dt.day_name()

# Count rows for each weekday
weekday_counts = TOP1['weekday'].value_counts().reset_index()
weekday_counts.columns = ['weekday', 'count']

# Calculate total number of rows
total_rows = len(TOP1)

# Calculate percentage of rows in weekend (Saturday and Sunday)
weekend_rows = weekday_counts[weekday_counts['weekday'].isin(['Saturday', 'Sunday'])]['count'].sum()
percent_weekend = (weekend_rows / total_rows) * 100

# Calculate percentage of rows in weekdays (Monday to Friday)
weekday_rows = weekday_counts[weekday_counts['weekday'].isin(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'])]['count'].sum()
percent_weekday = (weekday_rows / total_rows) * 100

print(f"Percentage of rows in weekend: {percent_weekend:.2f}%")
print(f"Percentage of rows in weekdays: {percent_weekday:.2f}%")

## 8.3: Distribution of Counts by Weekdays

In [ ]:
# Assuming TOP1 is already loaded with the data
# Convert year, month, day columns to datetime format
TOP1['date'] = pd.to_datetime(TOP1[['year', 'month', 'day']])

# Extract weekday names
TOP1['weekday'] = TOP1['date'].dt.day_name()

# Count rows for each weekday
weekday_counts = TOP1['weekday'].value_counts().reset_index()
weekday_counts.columns = ['weekday', 'count']

# Sort weekdays in chronological order
weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
weekday_counts['weekday'] = pd.Categorical(weekday_counts['weekday'], categories=weekdays, ordered=True)
weekday_counts = weekday_counts.sort_values('weekday')

# Plotting with plotly express
fig = px.bar(weekday_counts, x='weekday', y='count', 
             color_discrete_sequence=['skyblue'],  # Set color to a single color
             labels={'count': 'Count of Records', 'weekday': 'Weekday'},
             title='Distribution of Records by Weekday',
             text='count')  # Add text labels based on the 'count' column

fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')  # Format text labels

fig.update_layout(xaxis={'categoryorder':'array', 'categoryarray': weekdays},
                  showlegend=False)  # Hide legend

fig.show(renderer='iframe')

## 8.4: Distribution of Counts by Weather Condition

In [ ]:
!pip install meteostat

In [ ]:
from datetime import datetime
from meteostat import Point, Daily, units
import pandas as pd

# Assuming TOP1 is already loaded with the data
# Create Point for New York City
nyc = Point(40.7128, -74.0060)

# Convert year, month, day columns to datetime format
TOP1['date'] = pd.to_datetime(TOP1[['year', 'month', 'day']])

# Define a function to fetch weather condition for a given date
def fetch_weather(date):
    start = date
    end = date
    data = Daily(nyc, start, end)
    data = data.convert(units.imperial)
    data = data.fetch()
    
    if not data.empty:
        # Assuming we infer conditions like so
        # Modify this part based on the actual available data and your inference rules
        if data.iloc[0]['prcp'] > 0:
            return 'Rainy'
        else:
            return 'Sunny'
    else:
        return None

# Fetch weather conditions for each date in TOP1 and create a new column 'weather_condition'
TOP1['weather_condition'] = TOP1['date'].apply(fetch_weather)

# Remove rows where weather condition is None (no data available)
TOP1 = TOP1.dropna(subset=['weather_condition'])

# Display the DataFrame with the new column
print(TOP1)

In [ ]:
!pip install meteostat plotly

In [ ]:
from datetime import datetime
from meteostat import Point, Daily, units

# Assuming TOP1 is already loaded with the data
# Create Point for New York City
nyc = Point(40.7128, -74.0060)

# Convert year, month, day columns to datetime format
TOP1['date'] = pd.to_datetime(TOP1[['year', 'month', 'day']])

# Define a function to fetch weather condition for a given date
def fetch_weather(date):
    start = date
    end = date
    data = Daily(nyc, start, end)
    data = data.convert(units.imperial)
    data = data.fetch()
    
    if not data.empty:
        if data.iloc[0]['prcp'] > 0:
            return 'Rainy'
        else:
            return 'Sunny'
    else:
        return None

# Fetch weather conditions for each date in TOP1 and create a new column 'weather_condition'
TOP1['weather_condition'] = TOP1['date'].apply(fetch_weather)

# Remove rows where weather condition is None (no data available)
TOP1 = TOP1.dropna(subset=['weather_condition'])

# Display the DataFrame with the new column
print(TOP1.head())

In [ ]:
TOP1

In [ ]:
import plotly.express as px

# Plotting the distribution of TOP1 based on weather conditions
fig = px.histogram(TOP1, x='weather_condition', 
                   labels={'weather_condition': 'Weather Condition', 'count': 'Number of Records'},
                   title='Distribution of Records by Weather Condition')

# Add count labels on bars
fig.update_traces(texttemplate='%{y}', textposition='outside')

# Show the plot
fig.show(renderer='iframe')

In [ ]:
# Calculate the counts of each weather condition
weather_counts = TOP1['weather_condition'].value_counts()

# Calculate the total number of rows
total_rows = len(TOP1)

# Calculate percentages
weather_percentages = (weather_counts / total_rows) * 100

# Display the percentages
print(weather_percentages)

# Alternatively, you can print them in a more readable format
for condition, percentage in weather_percentages.items():
    print(f"{condition}: {percentage:.2f}%")

In [ ]:
TOP1.to_csv('TOP1_output.csv', index=False)

## 8.5: Calculate Correlations

In [ ]:
# Assuming your DataFrame is named TOP1
# Create a mapping of weekdays to numbers
weekday_mapping = {
    'Monday': 0,
    'Tuesday': 1,
    'Wednesday': 2,
    'Thursday': 3,
    'Friday': 4,
    'Saturday': 5,
    'Sunday': 6
}

# Add a new column to the DataFrame for the encoded weekdays
TOP1['weekday_encoded'] = TOP1['weekday'].map(weekday_mapping)

# Calculate the correlation between count and weekday_encoded
correlation = TOP1['count'].corr(TOP1['weekday_encoded'])

print("Correlation between count and weekday:", correlation)

In [ ]:
correlation_hour_count = TOP1['count'].corr(TOP1['hour'])

print("Correlation between count and hour:", correlation_hour_count)

In [ ]:
# Assuming TOP1 is already loaded with the data
# Create Point for New York City
nyc = Point(40.7128, -74.0060)

# Convert year, month, day columns to datetime format
TOP1['date'] = pd.to_datetime(TOP1[['year', 'month', 'day']])

# Define a function to fetch weather condition for a given date
def fetch_weather(date):
    start = date
    end = date
    data = Daily(nyc, start, end)
    data = data.convert(units.imperial)
    data = data.fetch()
    
    if not data.empty:
        if data.iloc[0]['prcp'] > 0:
            return 1  # Rainy
        else:
            return 0  # Sunny
    else:
        return None

# Fetch weather conditions for each date in TOP1 and create a new column 'weather'
TOP1['weather'] = TOP1['date'].apply(fetch_weather)

# Remove rows where weather condition is None (no data available)
TOP1 = TOP1.dropna(subset=['weather'])

# Calculate trip counts per day
daily_trip_counts = TOP1.groupby('date').size().reset_index(name='trip_count')

# Merge the weather data with the trip counts
daily_trip_counts = daily_trip_counts.merge(TOP1[['date', 'weather']].drop_duplicates(), on='date', how='left')

# Calculate the correlation between trip_count and weather
correlation = daily_trip_counts['trip_count'].corr(daily_trip_counts['weather'])

print("Correlation between trip count and weather (1 for rainy, 0 for sunny):", correlation)